In [1]:
import os
import json

import turbodbc

import pandas as pd
import numpy as np

In [2]:
with open("nettobadshop.json") as f:
    js = json.load(f)

In [6]:
def create_connection_string_turbo(server, database):
    options = turbodbc.make_options(prefer_unicode=True)
    constr = 'Driver={ODBC Driver 13 for SQL Server};Server=' + \
        server + ';Database=' + database + ';Trusted_Connection=yes;'
    con = turbodbc.connect(connection_string=constr, turbodbc_options=options)
    return con

In [19]:
def save_to_float(x):
    try:
        return float(x)
    except TypeError:
        return np.nan

In [20]:
def transform_df(df):
    df["price"] = df["price"].apply(lambda x: save_to_float(x)).astype(float)
    df.columns = ['Kategorie', 'Bezeichnung', 'Artikelnummer', 'Preis', 'Lieferant']
    return df

In [21]:
df = pd.DataFrame(js).pipe(transform_df)

In [22]:
df.head()

,Kategorie,Bezeichnung,Artikelnummer,Preis,Lieferant
0,Einbaukosten,Einbaukosten Zwei- und Dreilocharmatur,6000015,51.50,None
1,Duravit,Auflegewaschtisch Philippe Starck 1 D. 53 cm,2147146,767.86,None
2,Schulwandbrunnen und Reihenwaschtische,"Waschtisch Romay Varello Breite 360 cm, Höhe ...",7412175,4213.26,None
3,Waschraummöbel,"Waschtrog-Unterbau zu Waschraumlösung, Breite ...",7338502,1278.44,Romay
4,Duravit,"Einbauwaschtisch Philippe Starck 3, 49 x 36 cm...",2147536,408.86,None


In [27]:
df['Bezeichnung'].to_clipboard()

In [23]:
query = """
set nocount on;

SELECT  [Artikelnummer]
      ,[Artikel Beschreibung]
	  ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[Lieferantennummer]
into #temp
FROM [AnalystCM].[dbo].[LOOKUP_ArtikelstammCRHT]
where Artikelgruppe between 5000 and 5999 and [Einmalartikel (J/N)] = 0 and [Artikel Status] = '02'

select idItemOrigin
,AvgPrice = Avg(Sales / case when Quantity = 0 then NULL else Quantity end)
,WAvgPrice = sum(Sales) / case when sum(Quantity) = 0 then NULL else sum(Quantity) end
into #avgprice
from crhbusadwh01.infopool.fact.sales
where date > dateadd(month,-12, getdate())
group by idItemOrigin

select [Artikelnummer]
      ,[Artikel Beschreibung]
	  ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[ArtikelnummerLieferant] = [Lieferantennummer]
	  ,Durchschnittspreis = AvgPrice
	  ,GDurchschnittspreis = WAvgPrice
from #temp t
	left join #avgprice a on t.Artikelnummer = a.idItemOrigin collate Latin1_General_CI_AS


drop table #temp
drop table #avgprice
"""

In [24]:
con = create_connection_string_turbo("CRHBUSADWH51", "Operations")
df_server = pd.read_sql(query, con)
df_server['Bruttoverkauf'] = df_server['Bruttoverkauf'].astype("float")
df_server['Standardkosten'] = df_server['Standardkosten'].astype("float")